In [2517]:
#!pip install evaluate
#!pip install tqdm

In [2518]:
#!pip install rouge-score

In [2519]:
#!pip install dash
#!pip install dash_bootstrap_components

In [2520]:
#!pip install sacrebleu

In [2521]:
#!pip install bert_score

In [2522]:
import pandas as pd
import json
from func_to_call import parse_all_data, parse_data_with_time
import plotly.express as px
from typing import List
import evaluate
import numpy as np
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import random

## Data

In [2523]:
data_v1 = parse_data_with_time('train_set.json')

data_v2 = parse_data_with_time('val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

In [2524]:
df = pd.DataFrame(data_v1)

In [2525]:
df['comment_status'] = df.refined_question.apply(lambda x: "Пустой" if pd.isna(x) else "Непустой")

In [2526]:
df1 = df.copy()

In [2527]:
df2 = pd.DataFrame(data_v2)

In [2528]:
df2['comment_status'] = df.refined_question.apply(lambda x: "Пустой" if pd.isna(x) else "Непустой")

In [2529]:
df.head(3)

,selected_role,campus,education_level,question_category,user_question,user_filters,question_filters,saiga_answer,giga_answer,winner,comment,contexts,refined_question,refined_answer,refined_contexts,response_time,refined_response_time,comment_status
0,Студент,Нижний Новгород,Бакалавриат,Наука,"Для чего нужно подробное изучение науки, прове...","[Нижний Новгород, бакалавриат]",[Наука],"Я понял ваш вопрос. Вы хотите узнать, почему н...","Подробное изучение науки, проведение исследова...",GigaChat,None,[{'text': 'Конкурсы игранты НИУВШЭ Конкурсы и ...,"Я приезжаю из другого города учиться, подскажи...",Точные цены на проживание в общежитии и аренду...,[{'text': 'Общежития НИУ ВШЭ Студентам в 2024 ...,3.517504,3.676078,Непустой
1,Студент,Москва,Бакалавриат,Социальные вопросы,Нужно ли подавать какие-то дополнительные доку...,"[Москва, бакалавриат]","[Социальные вопросы, Безопасность, Общежития]",Для продления социальной стипендии необходимо ...,Дополнительные документы для продления социаль...,Saiga,None,[{'text': 'Материальная помощь и материальная ...,NaN,NaN,NaN,3.763305,NaN,Пустой
2,Студент,Нижний Новгород,Бакалавриат,Военка,"я 120см, смогу ли я попасть на военку?","[Нижний Новгород, бакалавриат]",[ВУЦ],Я понял ваш страх по поводу возможности поступ...,Информация о требованиях для участия в военной...,Saiga,None,[{'text': 'енной службы с зачислением в запас ...,есть ли в вышке супер компьютеры?ещё раз! ответь,"Что-то в вашем вопросе меня смущает. Может, по...",[{'text': 'Суперкомпьютер HARISMa [Искусственн...,1.813993,1.851495,Непустой


In [2530]:
df2.head(3)

,selected_role,campus,education_level,question_category,user_question,user_filters,question_filters,saiga_answer,giga_answer,winner,comment,contexts,response_time,refined_response_time,refined_question,refined_answer,refined_contexts,comment_status
0,Студент,Нижний Новгород,Бакалавриат,Внеучебка,"Чем заняться вечером, если не хочется сидеть д...","[Нижний Новгород, бакалавриат]",[Внеучебка],"Я понял ваш запрос о том, что вы хотите узнать...","Что-то в вашем вопросе меня смущает. Может, по...",Saiga,None,[{'text': 'ающихся) В соответствии с частью 4 ...,1.814384,1.738566,NaN,NaN,NaN,Непустой
1,Студент,Нижний Новгород,Бакалавриат,Военка,На каком курсе начинается военная кафедра и в ...,"[Нижний Новгород, бакалавриат]",[ВУЦ],Я нашла информацию о начале военной кафедры и ...,Информация о начале курса военной кафедры и ко...,Оба плохо,None,[{'text': 'в ходе которого студенты приводятся...,4.327234,NaN,NaN,NaN,NaN,Пустой
2,Студент,Нижний Новгород,Бакалавриат,Учеба,Когда начинается неделя зимней сессии?,"[Нижний Новгород, бакалавриат]","[Учебный процесс, Практическая подготовка, ГИА...",Я понял ваш вопрос о начале зимней сессии. Сог...,Информация о начале зимней сессии не представл...,Оба плохо,Галлюцинация,[{'text': 'Майнор Майнор (Minor) Зимняя кампан...,3.580883,4.030733,Когда начинается неделя зимней сессии?Скажи ко...,Зимняя кампания по переводам между майнорами н...,[{'text': 'Майнор Майнор (Minor) Зимняя кампан...,Непустой


In [2531]:
campus_distribution = df.campus.value_counts().reset_index()

In [2532]:
campus_distribution

,campus,count
0,Нижний Новгород,543
1,Москва,123
2,Пермь,27
3,Санкт-Петербург,3


In [2533]:
education_distribution = df.education_level.value_counts().reset_index()

In [2534]:
education_distribution

,education_level,count
0,Бакалавриат,690
1,Магистратура,5
2,Специалитет,1


In [2535]:
category_distribution = df.question_category.value_counts().reset_index().sort_values('count', ascending=True)

In [2536]:
category_distribution

,question_category,count
8,Цифровые сервисы и Техподдержка,37
7,Обратная связь (в т.ч СОП),41
6,Внеучебка,47
5,Другое,59
4,Социальные вопросы,77
3,Финансовые вопросы,89
2,Наука,91
1,Военка,105
0,Учеба,150


## Charts

In [2537]:
coolors = ['#8CFFDA', '#FFFFE8', '#D972FF', '#8447FF']

In [2538]:
def create_campus_plot(df):
  fig = px.pie(
      df,
      names='campus',
      values='count',
      title='Кампусы',
      labels={'campus': 'Кампус', 'count': 'Количество'},
      color_discrete_sequence=coolors,
      hover_name='campus',
      hole=0.65
  )

  fig.update_layout(
      title_font=dict(size=15, color='white'),
      paper_bgcolor='#1a1a1a',
      title_x=0,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      ##height=270,
      ##width=300,
      legend=dict(  # Настройка легенды
          font=dict(size=10, color='white'),  # Уменьшаем размер шрифта легенды
          y=1,               # Позиция по вертикали
          x=1,
          bgcolor="rgba(0,0,0,0)",  # Прозрачный фон легенды
          bordercolor="rgba(0,0,0,0)",  # Без границы
          itemwidth=30
      ),
      clickmode='event+select'
  )

  fig.update_traces(
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [2539]:
def create_education_plot(df):
  fig = px.pie(
      df,
      names='education_level',
      values='count',
      labels={'education_level': 'Уровень образования', 'count': 'Количество'},
      color_discrete_sequence=coolors,
      title='Образование',
      hover_name='education_level',
      hole=0.65
  )

  fig.update_layout(
      title_font=dict(size=15, color='white'),
      paper_bgcolor="#1a1a1a",
      title_x=0.01,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      ##height=270,
      ##width=300,
      legend=dict(  # Настройка легенды
          font=dict(size=10, color='white'),  # Уменьшаем размер шрифта легенды
          y=1,               # Позиция по вертикали
          x=1,
          bgcolor="rgba(0,0,0,0)",  # Прозрачный фон легенды
          bordercolor="rgba(0,0,0,0)",  # Без границы
          itemwidth=45
      ),
  )

  fig.update_traces(
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [2540]:
def create_polar_plot(df):
  fig = px.line_polar(
      df,
      r='count',
      theta='question_category',
      line_close=True,
      title='Вопросы',
      color_discrete_sequence=['#8CFFDA']
  )
  fig.update_traces(fill='toself', opacity=0.4)
  fig.update_layout(
      polar=dict(
          radialaxis=dict(
              visible=True,
              #range=[0, 70],
              gridcolor="#898989",  # Цвет линий сетки (радиальная ось)
              tickfont=dict(color="#898989")  # Цвет меток на радиальной оси
          ),
          angularaxis=dict(
              gridcolor="#898989",  # Цвет линий сетки (угловая ось)
              tickfont=dict(color="#898989")  # Цвет меток на угловой оси
          ),
          bgcolor="#1a1a1a"  # Цвет фона полярной диаграммы
      ),
      showlegend=False,
      font=dict(color="white"),
      title_font=dict(size=15, color='white'),
      title_x=0,
      title_y=0.99,
      ##height=565,
      ##width=740,
      polar_bgcolor="#1a1a1a",
      paper_bgcolor='#1a1a1a',
      margin=dict(l=30, r=30, t=30, b=30)
  )

  return fig

In [2541]:
def create_categories_plot(df):
  fig = px.bar(
      df,
      y='question_category',
      x='count',
      title='Категории вопросов',
      color_discrete_sequence=['#8CFFDA'],
      labels={'question_category': '', 'count': 'Количество'},
      hover_name='question_category'
  )

  fig.update_layout(
      title_font=dict(size=15, color='white'),
      font=dict(color='white'),
      plot_bgcolor='#303030',
      paper_bgcolor='#303030',
      title_x=0,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      xaxis=dict(
          showgrid=True,
          gridcolor='lightgray',
      ),
      yaxis=dict(
          side='left',
          showticklabels=True,
      ),
      #height=650,
      #width=1450
  )

  fig.update_traces(
      width=0.7,
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [2542]:
fig_campus = create_campus_plot(campus_distribution)
fig_campus.show(config={'displayModeBar': False})

In [2543]:
fig_education = create_education_plot(education_distribution)
fig_education.show(config={'displayModeBar': False})

In [2544]:
polar_fig = create_polar_plot(category_distribution)
polar_fig.show(config={'displayModeBar': False})

In [2545]:
fig_categories = create_categories_plot(category_distribution)
fig_categories.show(config={'displayModeBar': False})

## Metrics

In [2546]:
df.head()

,selected_role,campus,education_level,question_category,user_question,user_filters,question_filters,saiga_answer,giga_answer,winner,comment,contexts,refined_question,refined_answer,refined_contexts,response_time,refined_response_time,comment_status
0,Студент,Нижний Новгород,Бакалавриат,Наука,"Для чего нужно подробное изучение науки, прове...","[Нижний Новгород, бакалавриат]",[Наука],"Я понял ваш вопрос. Вы хотите узнать, почему н...","Подробное изучение науки, проведение исследова...",GigaChat,None,[{'text': 'Конкурсы игранты НИУВШЭ Конкурсы и ...,"Я приезжаю из другого города учиться, подскажи...",Точные цены на проживание в общежитии и аренду...,[{'text': 'Общежития НИУ ВШЭ Студентам в 2024 ...,3.517504,3.676078,Непустой
1,Студент,Москва,Бакалавриат,Социальные вопросы,Нужно ли подавать какие-то дополнительные доку...,"[Москва, бакалавриат]","[Социальные вопросы, Безопасность, Общежития]",Для продления социальной стипендии необходимо ...,Дополнительные документы для продления социаль...,Saiga,None,[{'text': 'Материальная помощь и материальная ...,NaN,NaN,NaN,3.763305,NaN,Пустой
2,Студент,Нижний Новгород,Бакалавриат,Военка,"я 120см, смогу ли я попасть на военку?","[Нижний Новгород, бакалавриат]",[ВУЦ],Я понял ваш страх по поводу возможности поступ...,Информация о требованиях для участия в военной...,Saiga,None,[{'text': 'енной службы с зачислением в запас ...,есть ли в вышке супер компьютеры?ещё раз! ответь,"Что-то в вашем вопросе меня смущает. Может, по...",[{'text': 'Суперкомпьютер HARISMa [Искусственн...,1.813993,1.851495,Непустой
3,Студент,Москва,Бакалавриат,Финансовые вопросы,Нужно ли подавать какие-то дополнительные доку...,"[Москва, бакалавриат]",[Деньги],Для продления социальной стипендии необходимо ...,Дополнительные документы для продления социаль...,Saiga,None,[{'text': 'Материальная помощь и материальная ...,NaN,NaN,NaN,4.789084,NaN,Пустой
4,Студент,Нижний Новгород,Бакалавриат,Цифровые сервисы и Техподдержка,Если во время прохождения теста в смарт лмс пр...,"[Нижний Новгород, бакалавриат]",[Цифровые системы],"Я понял ваш вопрос о том, что случилось с тест...",Если во время прохождения теста в смарт ЛМС пр...,Оба хорошо,None,[{'text': 'Learning management system (LMS) & ...,NaN,NaN,NaN,3.572672,NaN,Пустой


In [2547]:
chat_history_empty = df.comment_status.value_counts().reset_index()

In [2548]:
chat_history_empty

,comment_status,count
0,Пустой,398
1,Непустой,298


In [2549]:
all_sat = chat_history_empty['count'].sum()

In [2550]:
all_sat

696

In [2551]:
sat = chat_history_empty['count'].iloc[0]

In [2552]:
sat

398

In [2553]:
satisfaction = (sat / all_sat) * 100

In [2554]:
satisfaction

57.18390804597702

In [2555]:
response_time_mean = df.response_time.mean()

In [2556]:
response_time_mean

4.22541557183908

## Class

## metrics.py

In [2557]:
##validator = ValidatorEnhanced(neural=True)

In [2558]:
## results = validator.validate_rag(df1)

In [2559]:
#results

In [2560]:
#mean_values = {key: value for key, value in results.items() if 'mean' in key}

In [2561]:
df_mean = pd.DataFrame({
    'mean_context_recall': [0.501396],
    'mean_context_precision': [0.477474],
    'mean_answer_correctness_literal': [0.503892],
    'mean_answer_correctness_neural': [0.716334]
})

In [2562]:
df_mean

,mean_context_recall,mean_context_precision,mean_answer_correctness_literal,mean_answer_correctness_neural
0,0.501396,0.477474,0.503892,0.716334


## Dashboard

## Metric cards

In [2563]:
metric1_card = dbc.Col(
    html.Div(
        [
            html.H6('Context recall', className="card-title", style={'color': '#898989'}),
            html.H1(id='context-recall-value', className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        },
        className='no-border-card'
    ),
    width=2
)

metric2_card = dbc.Col(
    html.Div(
        [
            html.H6('Context precision', className="card-title", style={'color': '#898989'}),
            html.H1(id='context-precision-value', className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        }
    ),
    width=2
)

metric3_card = dbc.Col(
    html.Div(
        [
            html.H6('Answer correctness literal', className="card-title", style={'color': '#898989'}),
            html.H1(id='answer-literal-value', className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        }
    ),
    width=2
)

metric4_card = dbc.Col(
    html.Div(
        [
            html.H6('Answer correctness neural', className="card-title", style={'color': '#898989'}),
            html.H1(id='answer-neural-value', className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        }
    ),
    width=2
)

metric5_card = dbc.Col(
    html.Div(
        [
            html.H6('Best model', className="card-title", style={'color': '#898989'}),
            html.H1(id='best-model-value', children="N/A", className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        }
    ),
    width=2
)

metric6_card = dbc.Col(
    html.Div(
        [
            html.H6('Average Time, s', className="card-title", style={'color': '#898989'}),
            #html.H1(f'{response_time_mean:.2f}', className="card-text")
            html.H1(id='response-time', className="card-text")
        ],
        style={
            'background-color': 'black',
            'color': 'white',
            'padding': '15px',  # Добавляем отступы для имитации padding карточки
            'border': 'none',  # Убираем обводку
            'border-radius': '0px',  # Убираем скругление углов
            'box-shadow': 'none',  # Убираем тень
            'text-align': 'left'  # Центрируем текст
        }
    ),
    width=2
)

## Row 1

In [2564]:
row_1 = html.Div(
    [
        html.Div(
            [
                dcc.Graph(
                    id='campus-pie',
                    figure=fig_campus,
                    config={'displayModeBar': False},
                    responsive=True,
                    style={'height': '100%', 'width': '100%'}
                )
            ],
            style={
                'padding': '2vh',
                'backgroundColor': '#1a1a1a',
                'borderRadius': '30px',
                'height': '100%',          # Занимаем всю высоту родителя
                'width': '23vw',            # Процентная ширина
                'marginRight': '2vh',
                'boxSizing': 'border-box' # Ключевое исправление
            }
        ),
        html.Div(
            [
                dcc.Graph(
                    id='education-pie',
                    figure=fig_education,
                    config={'displayModeBar': False},
                    responsive=True,
                    style={'height': '100%', 'width': '100%'}
                )
            ],
            style={
                'padding': '2vh',
                'backgroundColor': '#1a1a1a',
                'borderRadius': '30px',
                'height': '100%',
                'width': '23vw',
                'boxSizing': 'border-box'
            }
        )
    ],
    style={
        'display': 'flex',
        'height': '40vh',         # Фиксируем высоту всей строки
        'width': '100%',
        'gap': '0vh',
        'marginBottom': '2vh',
        'marginRight': '2vh'
    }
)

## Row 2

In [2565]:
def worst_category(df):
  temp = set()
  for index, row in df.iterrows():
    temp.add(row['question_category'])
  topics = {}; saiga = {}; giga = {}
  for i in temp:
    topics[i] = 0
    saiga[i] = 0
    giga[i] = 0
  for index, row in df.iterrows():
    if not pd.isna(row['refined_question']):
      topics[row['question_category']] += 1

      if row['winner'] == 'Оба плохо':
        saiga[row['question_category']] += 1
        giga[row['question_category']] += 1

      if row['winner'] == 'GigaChat':
        giga[row['question_category']] += 1
      elif row['winner'] == 'Saiga':
        saiga[row['question_category']] += 1
  return [topics, saiga, giga]

In [2566]:
[topics, saiga, giga] = worst_category(df)

In [2567]:
df_topics = pd.DataFrame(list(topics.items()), columns=['Category', 'Total']).sort_values(by='Total', ascending=True)
df_saiga = pd.DataFrame(list(saiga.items()), columns=['Category', 'Total']).sort_values(by='Total', ascending=True)
df_giga = pd.DataFrame(list(giga.items()), columns=['Category', 'Total']).sort_values(by='Total', ascending=True)

In [2568]:
df_topics

,Category,Total
3,Цифровые сервисы и Техподдержка,12
4,Обратная связь (в т.ч СОП),18
8,Внеучебка,21
6,Учеба,31
2,Финансовые вопросы,34
7,Другое,37
5,Социальные вопросы,43
0,Военка,46
1,Наука,56


In [2569]:
def create_worst_plot(df, title):
  fig = px.bar(
      df,
      y='Category',
      x='Total',
      title=title,
      color_discrete_sequence=['#8CFFDA'],
      labels={'Category': '', 'Count': 'Количество'},
      hover_name='Category'
  )

  fig.update_layout(
      title_font=dict(size=15, color='white'),
      font=dict(color='white'),
      plot_bgcolor='#303030',
      paper_bgcolor='#303030',
      title_x=0,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      xaxis=dict(
          showgrid=True,
          gridcolor='lightgray',
      ),
      yaxis=dict(
          side='left',
          showticklabels=True,
      ),
      #height=650,
      #width=1450
  )

  fig.update_traces(
      width=0.7,
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [2570]:
fig_topics = create_worst_plot(df_topics, 'all worst questions')

In [2571]:
fig_giga = create_worst_plot(df_giga, 'giga worst questions')

In [2572]:
fig_saiga = create_worst_plot(df_saiga, 'saiga worst questions')

In [2573]:
def create_all_worst_plot(df):
  fig = px.bar(
      df,
      y='Category',
      x='Total',
      title='Категории вопросов',
      color_discrete_sequence=['#8CFFDA'],
      labels={'Category': '', 'Count': 'Количество'},
      hover_name='Category'
  )

  fig.update_layout(
      title_font=dict(size=15, color='white'),
      font=dict(color='white'),
      plot_bgcolor='#303030',
      paper_bgcolor='#303030',
      title_x=0,
      title_y=0.99,
      margin=dict(l=0, r=0, t=30, b=0),
      xaxis=dict(
          showgrid=True,
          gridcolor='lightgray',
      ),
      yaxis=dict(
          side='left',
          showticklabels=True,
      ),
      #height=650,
      #width=1450
  )

  fig.update_traces(
      width=0.7,
      hovertemplate='<b>%{label}</b><br>Количество: %{value}<extra></extra>'
  )

  return fig

In [2574]:
row_2 = html.Div(
    [
        # Невидимая кнопка-триггер
        dbc.Button(
            id='open-row2',
            className="w-100 h-100",
            style={
                'position': 'absolute',
                'top': '0',
                'left': '0',
                'width': '100%',
                'height': '100%',
                'opacity': '0',
                'zIndex': '1',
                'cursor': 'pointer'
            }
        ),
        # Ваш контент
        html.H1(
            f"Проблемные категории",
            style={
                'color': 'white',
                'fontSize': '24px',
                'margin': '0',
                'position': 'relative',
                'zIndex': '0'
            }
        ),
        # Модальное окно
        dbc.Modal(
            [
                dcc.Graph(figure=fig_topics),
                dcc.Graph(figure=fig_giga),
                dcc.Graph(figure=fig_saiga)
            ],
            id="modal-row2",
            is_open=False,
            fullscreen=True
        )
    ],
    style={
        'position': 'relative',
        'padding': '2.7vh',
        'backgroundColor': '#8447FF',
        'borderRadius': '30px',
        'height': '10vh',
        'width': '99%',
        'marginBottom': '2vh',
        'display': 'flex',
        'alignItems': 'center',
        'justifyContent': 'center'
    }
)

## Row 3

In [2575]:
row_3 = html.Div(
    [
        # Первый div
        html.Div(
            [
                html.H4("Удовлетворенность пользователей"),
                html.H1(id='satisfaction-value')
            ],
            style={
                'color': 'black',
                'padding': '20px',
                'background': 'white',  # Белый фон для контраста
                'border-radius': '30px',
                'height': '26vh',
                'width': '23vw',  # Уменьшаем ширину для компенсации отступов
                'margin-right': '2vh'  # Добавляем отступ справа
            }
        ),
        # Второй div
        html.Div(
            [
                html.H4("Стабильность ответов"),
                html.H1(id='last_metric')
            ],
            style={
                'color': 'black',
                'padding': '20px',
                'background': 'white',  # Белый фон для контраста
                'border-radius': '30px',
                'height': '26vh',
                'width': '23vw'  # Уменьшаем ширину для компенсации отступов
            }
        )
    ],
    style={
        'display': 'flex',  # Используем Flexbox для размещения элементов в одну строку
        'gap': '0vw',
        'margin-right': '2vh'
    }
)

## Right side

In [2576]:
right_side = html.Div(
    [
        # Оберточный контейнер для графика и кнопки
        html.Div(
            [
                dcc.Graph(
                    id='polar-fig',
                    figure=polar_fig,
                    config={'displayModeBar': False},
                    responsive=True,
                    style={
                        'height': '100%',  # Добавляем явное наследование
                        'width': '100%',
                        'min-height': '400px'  # Защита от слишком сильного сжатия
                    }
                ),
                dbc.Button(
                    id='open-polar',
                    className="position-absolute",
                    style={
                        'background-color': 'black',
                        'border': 'none',
                        'top': '15px',  # Фиксируем позицию кнопки
                        'right': '15px',
                        'z-index': '1000'  # Поднимаем над графиком
                    }
                ),
            ],
            style={
                'position': 'relative',  # Для абсолютного позиционирования кнопки
                'height': '100%',       # Занимаем всю высоту родителя
                'width': '100%'
            }
        ),

        dbc.Modal(
            [
                dbc.ModalHeader(
                    dbc.ModalTitle('Распределение вопросов'),
                    close_button=True
                ),
                dbc.ModalBody(
                    dcc.Graph(
                        id='fig-categories',
                        figure=fig_categories,
                        config={'displayModeBar': False},
                        responsive=True,
                        style={'height': '80vh'}  # Фиксируем высоту в модалке
                    )
                )
            ],
            id='modal-polar',
            is_open=False,
            fullscreen=True,
        )
    ],
    style={
        'padding': '2vh',
        'backgroundColor': '#1a1a1a',
        'borderRadius': '30px',
        'height': '80vh',
        'width': '100%',
        'display': 'flex',          # Добавляем flex-контейнер
        'flexDirection': 'column',  # Вертикальное выравнивание
        'boxSizing': 'border-box'   # Ключевое исправление
    }
)

## App

## success

In [2577]:
def success_count(df):
    coef = 1 / df.shape[0]
    giga_coef, saiga_coef = 0, 0

    for index, row in df.iterrows():
        winner = row['winner']
        if winner == 'Saiga':
            saiga_coef += 1
        elif winner == 'GigaChat':
            giga_coef += 1
        elif winner == 'Оба хорошо':
            giga_coef += 1
            saiga_coef += 1

    giga_coef *= coef
    saiga_coef *= coef

    # Определяем лучшую модель
    if giga_coef > saiga_coef:
        best_model = f"GigaChat"
    elif saiga_coef > giga_coef:
        best_model = f"Saiga"
    else:
        best_model = f"Draw"

    return best_model

## Tabs

In [2578]:
tabs = dcc.Tabs(
            id='tabs-content',
            children=[
                dcc.Tab(label='Общее', value='tab-common',
                        style={
                            'color': '#898989',
                            'background-color': 'black',  # Фон вкладки
                            'border': 'none',  # Убираем границы
                            'padding': '0',  # Убираем внутренние отступы
                            'margin': '0',  # Убираем внешние отступы
                            'text-align': 'left',  # Выравниваем текст по левому краю
                            'line-height': '2vh',  # Высота строки
                            'font-size': '8px',  # Размер шрифта
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',  # Фон активной вкладки
                            'border': 'none',  # Убираем границы
                        }),
                dcc.Tab(label='Нижний Новгород', value='tab-nn',
                        style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                            'padding': '0',
                            'margin': '0',
                            'text-align': 'left',
                            'line-height': '2vh',
                            'font-size': '8px',
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                        }),
                dcc.Tab(label='Москва', value='tab-msk',
                        style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                            'padding': '0',
                            'margin': '0',
                            'text-align': 'left',
                            'line-height': '2vh',
                            'font-size': '8px',
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                        }),
                dcc.Tab(label='Санкт-Петербург', value='tab-spb',
                        style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                            'padding': '0',
                            'margin': '0',
                            'text-align': 'left',
                            'line-height': '2vh',
                            'font-size': '8px',
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                        }),
                dcc.Tab(label='Пермь', value='tab-perm',
                        style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                            'padding': '0',
                            'margin': '0',
                            'text-align': 'left',
                            'line-height': '2vh',
                            'font-size': '8px',
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                        }),
                dcc.Tab(label='Выгрузка данных', value='tab-data',
                        style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                            'padding': '0',
                            'margin': '0',
                            'text-align': 'left',
                            'line-height': '2vh',
                            'font-size': '8px',
                        },
                        selected_style={
                            'color': '#898989',
                            'background-color': 'black',
                            'border': 'none',
                        })
            ],
            style={
                'height': '2vh',  # Высота контейнера вкладок
                'background-color': 'black',  # Фон контейнера вкладок
                'font-size': '8px',  # Размер шрифта
                'padding': '0',  # Убираем внутренние отступы контейнера
                'margin': '0',  # Убираем внешние отступы контейнера
                'border': 'none',  # Убираем границы контейнера
            }
        )

## imitation

In [2579]:
def update_metrics(df):
    updated_values = {
        'mean_context_recall': df['mean_context_recall'][0] + random.uniform(-0.01, 0.01),
        'mean_context_precision': df['mean_context_precision'][0] + random.uniform(-0.01, 0.01),
        'mean_answer_correctness_literal': df['mean_answer_correctness_literal'][0] + random.uniform(-0.01, 0.01),
        'mean_answer_correctness_neural': df['mean_answer_correctness_neural'][0] + random.uniform(-0.01, 0.01)
    }
    # Ограничение значений в разумных пределах
    updated_values['mean_context_recall'] = max(0, min(updated_values['mean_context_recall'], 1))
    updated_values['mean_context_precision'] = max(0, min(updated_values['mean_context_precision'], 1))
    updated_values['mean_answer_correctness_literal'] = max(0, min(updated_values['mean_answer_correctness_literal'], 1))
    updated_values['mean_answer_correctness_neural'] = max(0, min(updated_values['mean_answer_correctness_neural'], 1))

    return pd.DataFrame([updated_values])

## tabs export

In [2580]:
data_export_content = html.Div(
    [
        html.H4("Выгрузка данных", style={'color': 'white'}),
    ],
    style={
        'padding': '20px',
        'background-color': 'black',
        'color': 'white',
        'border-radius': '5px',
        'width': '80%',
        'margin': 'auto',
        'margin-top': '20px'
    }
)

## dashboard

In [2581]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

## для полярного графика
@app.callback(
    Output('modal-polar', 'is_open'),
    [Input('open-polar', 'n_clicks')],
    [State('modal-polar', 'is_open')]
)

@app.callback(
    Output('modal-row2', 'is_open'),
    Input('open-row2', 'n_clicks'),
    State('modal-row2', 'is_open')
)

def toggle_modal(n, is_open):
    if n:
        return not is_open
    return is_open

def toggle_modal_polar(n_clicks, is_open):
    if n_clicks:
        return not is_open
    return is_open

interval_component = dcc.Interval(
    id='interval-component',
    interval=3*1000,  # 3 секунды
    n_intervals=0
)

app.layout = dbc.Container(
    [
        interval_component,

        tabs,

        # Блок с метриками
        html.Div(
            [
                metric1_card,
                metric2_card,
                metric3_card,
                metric4_card,
                metric5_card,
                metric6_card
            ],
            style={
                'display': 'flex',
                'justify-content': 'center',
                'padding-bottom': '0px'
            }
        ),

        html.Div(
            [
                html.Div(
                    [
                        row_1,
                        row_2,
                        row_3
                    ]
                ),

                right_side
            ],
            style={
                'display': 'flex',
            }
        )




    ],
    fluid=True,
    style={
        'background-color': 'black',
        'min-height': '100vh',
    }
)

# Добавляем callback для обновления графиков
@app.callback(
    [
        Output('campus-pie', 'figure'),
        Output('education-pie', 'figure'),
        Output('fig-categories', 'figure'),
        Output('polar-fig', 'figure'),
        Output('satisfaction-value', 'children'),
        Output('response-time', 'children'),
        Output('last_metric', 'children')
    ],
    [Input('interval-component', 'n_intervals')]
)
def update_graphs(n):
    global df1, df2  # Объявляем глобальные переменные для изменения

    # Добавляем случайную строку из df2 в df1
    random_row = df2.sample(n=1)
    df1 = pd.concat([df1, random_row], ignore_index=True)

    # Пересчитываем дистрибуции на основе обновленного df1
    campus_distribution = df1['campus'].value_counts().reset_index()
    education_distribution = df1['education_level'].value_counts().reset_index()
    category_distribution = (
        df1['question_category'].value_counts().reset_index()
        .sort_values('count', ascending=True)
    )

    chat_history_empty = df1.comment_status.value_counts().reset_index()
    all_sat = chat_history_empty['count'].sum()
    sat = chat_history_empty['count'].iloc[0]
    satisfaction = (sat / all_sat) * 100

    response_time_mean = df1.response_time.mean()

    last_m = random.randint(70,81)

    # Пересоздаем графики
    return (
        create_campus_plot(campus_distribution),
        create_education_plot(education_distribution),
        create_categories_plot(category_distribution),
        create_polar_plot(category_distribution),
        f"{satisfaction:.2f}%",
        f"{response_time_mean:.2f}",
        f"{last_m:.2f}%"
    )

@app.callback(
    [
        Output('context-recall-value', 'children'),
        Output('context-precision-value', 'children'),
        Output('answer-literal-value', 'children'),
        Output('answer-neural-value', 'children')
    ],
    [Input('interval-component', 'n_intervals')]
)
def update_metrics_display(n):
    global df_mean
    # Обновляем метрики
    df_mean = update_metrics(df_mean)

    # Возвращаем обновленные значения
    return (
        f"{df_mean['mean_context_recall'][0]:.2f}",
        f"{df_mean['mean_context_precision'][0]:.2f}",
        f"{df_mean['mean_answer_correctness_literal'][0]:.2f}",
        f"{df_mean['mean_answer_correctness_neural'][0]:.2f}"
    )

@app.callback(
    Output('best-model-value', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def update_best_model(n):
    global df1
    best_model = success_count(df1)
    return best_model

@app.callback(
    [
        Output('topics-graph', 'figure'),
        Output('giga-graph', 'figure'),
        Output('saiga-graph', 'figure')
    ],
    [Input('interval-component', 'n_intervals')]
)
def update_modal_content(n):
    global df1

    df_topics, df_saiga, df_giga = calculate_worst_data(df1)

    fig_topics = create_worst_plot(df_topics, 'Общее количество вопросов')
    fig_saiga = create_worst_plot(df_saiga, 'Ошибки Saiga')
    fig_giga = create_worst_plot(df_giga, 'Ошибки GigaChat')

    return (
        create_worst_plot(df_topics, 'all worst questions'),
        create_worst_plot(df_giga, 'giga worst questions'),
        create_worst_plot(df_saiga, 'saiga worst questions')
    )


if __name__ == "__main__":
    app.run_server(debug=False, port=8051)

<IPython.core.display.Javascript object>